<a href="https://colab.research.google.com/github/tomshannon1/explore-latent-space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Latent Space of GAN Models

This notebook allows you to explore the latent space of GAN models that you have to trained. Through this process, you will be able find the directions of traversal to generate images with your desired traits.  

A GPU is required to run this so it's easiest to run on Google Colab with GPU or TPU enabled. To do this, click on *Runtime* and then *Change runtime type* to set your instance to GPU or TPU.


In [ ]:
# Mount your Goolge Drive to your Google Colab instance
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Next, clone StyleGAN2 from GitHub.

In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git

# Run StyleGAN2 From Python Code

Add the StyleGAN folder to the Python path to import ddnlib - which is a library required for importing your StyleGAN models. The code below is based on code from NVidia. This will generate images from your model.

In [4]:
import sys
sys.path.insert(0, "/content/stylegan2")
import dnnlib

In [5]:
# Copyright (c) 2019, NVIDIA Corporation. All rights reserved.
#
# This work is made available under the Nvidia Source Code License-NC.
# To view a copy of this license, visit
# https://nvlabs.github.io/stylegan2/license.html

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys

import pretrained_networks

#----------------------------------------------------------------------------

def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

def generate_images(Gs, seeds, truncation_psi, prefix):
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
        rnd = np.random.RandomState(0)
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
        path = f"/content/{prefix}-{seed_idx+1}.png"
        PIL.Image.fromarray(images[0], 'RGB').save(path)


In [ ]:
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'

# Load your Generator model
network_pkl = '/content/drive/My Drive/network-snapshot-000722.pkl'
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
vector_size = Gs.input_shape[1:][0]

# Explore Many GANs

The first thing you can do is generate many different GAN's randomly. From this you can find different traits that you are looking for. 

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

seed = 9200

for i in range(100):
  z = seed + i
  print(z)
  seeds = expand_seed([z], vector_size)
  generate_images(Gs, seeds, 0.5, "image")
  img = cv2.imread('/content/image-1.png')   
  cv2_imshow(img) 

# Modify Single GAN

Given one of the seed values, you can choose one to look at individually. We can modify the distribution by tweaking the truncation_psi. This effectively truncates the z vector by resampling the values with magnitude above a chosen threshold, which leads to improvement in individual sample quality at the cost of reduction in overall sample variety.

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow
import numpy as np

seeds = expand_seed([9208], vector_size)

for i in np.linspace(0.0, 1.0, 20):
  generate_images(Gs, seeds, i, "image")
  img = cv2.imread('/content/image-1.png')   
  cv2_imshow(img) 

# Explore Elements of Vector

This section of code lets you explore sections of the latent vector for a given seen by increments to look for desired traits. For more fine grained transitions you can decrease the increment. This distribution of images will later be compiled to an animation.

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

i = 0

while i<500:
  seeds = expand_seed([9208], vector_size)
  seeds[0][0][i:i+10] = -2
  print(i)
  generate_images(Gs, seeds, 0.5, "image")
  img = cv2.imread('/content/image-1.png')  
  cv2.imwrite('/content/image-%d.png' % i, img) 
  cv2_imshow(img) 
  i+=10

In [ ]:
# This is used to get the filenames of the images we just produced above
from os import listdir
from os.path import isfile, join
mypath = "/content"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [50]:
# This will compile these images into an animated GIF showing how we moved through the latent space
# Change the filename to your desired name and it will be saved in your Google Drive
import imageio
images = []

for filename in onlyfiles:
    images.append(imageio.imread(mypath + "/" + filename))
imageio.mimsave('/content/drive/My Drive/seed-9208.gif', images)